In [150]:
import pandas as pd
import numpy as np
from geopy.geocoders import OpenCage
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [151]:
df = pd.read_csv('../data/plane_crash_coordinatesV2.csv', encoding='unicode_escape', index_col = 0)
df.replace("''", np.nan, inplace=True)
df.dropna(subset = ['Latitude'], inplace=True)
df['Location'] = df['Location'].str.replace('Near', '').str.replace('Off', '').str.replace('Over the', '').str.strip()

In [152]:
def extract_country(location):
    parts = location.split(', ')
    if len(parts) > 1:
        return parts[-1]
    else:
        return location

# Apply the function to the 'location' column and store the result in a new column 'country'
df['Country'] = df['Location'].apply(extract_country)

# Define a list of all US states
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
    'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
    'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]
# Replace US states with 'United States' in the 'Country' column
df['Country'] = df['Country'].apply(lambda x: 'United States' if x in us_states else x)
df['Country'] = df['Country'].replace('England', 'United Kingdom')
df['Country'] = df['Country'].replace('Bulgeria', 'Bulgaria')

In [153]:
# Api key
api_key = "b7b37c7b2c7f49f69d03541f1abd4555"

In [154]:
def get_country_name(lat, lon, api_key):
    geolocator = OpenCage(api_key=api_key)
    
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        country = location.raw['components']['country']
        return country
    except GeocoderTimedOut:
        # Handle timeouts or errors as needed
        return None


In [156]:
#geolocator = Nominatim(user_agent="your_app_name")  # Replace with your app name

def get_country(latitude, longitude):
    s = 0
    hej = "your_name"+str(s)
    geolocator = Nominatim(user_agent=hej)
        
    location = None
    try:
        location = geolocator.reverse((latitude, longitude), language='en')
        s += 1
        
    except GeocoderTimedOut:
        print("Geocoding request timed out. Please try again.")

    if location:
        address = location.raw.get('address', {})
        country = address.get('country', '')
        return country
    else:
        return None
        


# Create a new 'country' column
df['Country coordinates'] = df.apply(lambda row: get_country(row['Latitude'], row['Longitude']), axis=1)


In [157]:
# Create a new column based on comparison
df['Location_Status'] = df.apply(lambda row: 'correct location' if row['Country'] == row['Country coordinates'] else 'not correct location', axis=1)

In [158]:
np.unique(df['Location_Status'], return_counts=True)


(array(['correct location', 'not correct location'], dtype=object),
 array([3083, 1435]))

In [159]:
df.to_csv('../data/plane_crash_coordinatesV5.csv')
